# Subset Selection
## CMSE 381 - Fall 2023
## Oct 13,  2023. Lecture 17



In [3]:
# Everyone's favorite standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [4]:
# First, we're going to do all the data loading we've had for a while for this data set
auto = pd.read_csv('Auto.csv')
auto = auto.replace('?', np.nan)
auto = auto.dropna()
auto.horsepower = auto.horsepower.astype('int')

#this shuffles my data set in advance so that i don't need to worry about it later 
auto = auto.sample(frac=1).reset_index(drop=True)


auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,14.0,8,318.0,150,4237,14.5,73,1,plymouth fury gran sedan
1,24.0,4,119.0,97,2545,17.0,75,3,datsun 710
2,33.0,4,105.0,74,2190,14.2,81,2,volkswagen jetta
3,11.0,8,318.0,210,4382,13.5,70,1,dodge d200
4,14.0,8,440.0,215,4312,8.5,70,1,plymouth fury iii


Let's try to run subset selection on the `auto` data set! We're going to use `cylinders`, `horsepower`, `weight`, and `acceleration` to predict `mpg`. 

In [5]:
inputvars = ['cylinders','horsepower','weight', 'acceleration']

The first tool we are going to use is the `itertools` package, which gives us a way to get subsets of whatever size we want using the `combinations` command.  

In [6]:
from itertools import combinations

The weird thing is it's an iterator, so if I just try to print out what I want, it's not helpful to me. 

In [7]:
combinations(inputvars,2)

But if I use it in a for loop it does what I want!

In [8]:
for x in combinations(inputvars,2):
    print(x)

('cylinders', 'horsepower')
('cylinders', 'weight')
('cylinders', 'acceleration')
('horsepower', 'weight')
('horsepower', 'acceleration')
('weight', 'acceleration')


Here's some code stolen from the last few days to run linear regression on a subset of the input variables. 

In [9]:
def myscore(df,listofvars, outputvar = 'mpg'):
    X = df[list(listofvars)]
    y = df[outputvar]
    
    #build linear regression model
    model = LinearRegression()
    

    #use 5-fold CV to evaluate model
    scores = cross_val_score(model, X,y, 
                             scoring='neg_mean_squared_error',
                             cv=5)

    #view mean absolute error
    return np.average(np.absolute(scores))
    

myvars = ('cylinders', 'acceleration')
myscore(auto,myvars)

24.40314626592509


&#9989; **<font color=red>Do this:</font>** Modify the code below as follows: 
- Set up two nested for loops to get every size $p = \{1,\cdots,4\}$ subset of my list of variables I want to use
- For each of these subsets, use the `myscore` function to get the k-fold CV error.
- Append it into a data frame as shown


In [10]:
myvars = []
myscores = []

for p in range(1, len(inputvars)+1):
    for subset in combinations(inputvars, p):
        score = myscore(auto, subset)
        myvars.append(subset)
        myscores.append(score)
        
myResults = pd.DataFrame({'Vars':myvars, 'Score':myscores})
myResults

,Vars,Score
0,"(cylinders,)",24.263435
1,"(horsepower,)",24.157741
2,"(weight,)",18.866672
3,"(acceleration,)",50.305412
4,"(cylinders, horsepower)",21.115910
5,"(cylinders, weight)",18.672779
6,"(cylinders, acceleration)",24.403146
7,"(horsepower, weight)",18.021227
8,"(horsepower, acceleration)",22.791596
9,"(weight, acceleration)",18.443104


We got all our main subsets, we're just missing a null model. This is the model that predicts the sample mean `mpg` for any input data. 

&#9989; **<font color=red>Do this:</font>** What is the MSE on our data set if we just predict the mean for every data point? Add this entry to your `myResults` data frame

*Hint: you can get a numpy array with every entry being the same output by using the `np.full` command.*

In [11]:
mean_prediction = auto['mpg'].mean()

null_predictions = np.full(len(auto), mean_prediction)

myscore = mean_squared_error(auto['mpg'], null_predictions)

In [12]:
newresult = pd.DataFrame({'Vars':['empty'], 'Score':[myscore]})

myResults = pd.concat([newresult,myResults],ignore_index = True)

# If you print it out now, you should have 16 models scored
myResults

,Vars,Score
0,empty,60.762738
1,"(cylinders,)",24.263435
2,"(horsepower,)",24.157741
3,"(weight,)",18.866672
4,"(acceleration,)",50.305412
5,"(cylinders, horsepower)",21.115910
6,"(cylinders, weight)",18.672779
7,"(cylinders, acceleration)",24.403146
8,"(horsepower, weight)",18.021227
9,"(horsepower, acceleration)",22.791596


&#9989; **<font color=red>Do this:</font>** What is the minimum score over all subsets? Which model makes it happen? The `idxmin` command will likely be useful for this. 

In [14]:
best_model_idx = myResults['Score'].idxmin()
best_model = myResults.loc[best_model_idx]

print(f"The minimum MSE score is {best_model['Score']:.4f}")
print(f"Achieved by using variables: {best_model['Vars']}")

The minimum MSE score is 17.9960
Achieved by using variables: ('cylinders', 'horsepower', 'weight')


## Stretch project

Have some free time? 

&#9989; **<font color=red>Strech project:</font>** Figure out how to do this for forward selection. 

- First, try to write pseudo code for how to search through these subsets. 
- Then if you still have time, see if you can get it working on this set of four variables. 

In [ ]:
# Your code here #



-----
### Congratulations, we're done!
Written by Dr. Liz Munch, Michigan State University

<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.